<a href="https://colab.research.google.com/github/togakyo/numpy-100/blob/master/blockchain_sample_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from dataclasses import dataclass


In [0]:
@dataclass
class Transaction:
    sender_address: int
    receiver_address: int
    value: float

In [0]:
@dataclass
class Wallet:
    address: int
    def send(self, receiver_address, value):
        return Transaction(self.address, receiver_address, value)

In [0]:
Ledger=list

In [0]:
alice=Wallet(1) # create Alice's wallet
bob=Wallet(2)   # create Bob's wallet
ledger=Ledger() # create Bank's ledger

In [0]:
transaction=alice.send(bob.address, 5) # Alice send `5` to Bob
ledger.append(transaction)

In [0]:
import json
import dataclasses as dc

In [0]:
@dataclass
class Transaction:
    sender_address: str
    receiver_address: str
    value: float
    sign: str = None
        
    def str_data(self) -> str:
        # return the str of data without `sign`
        d=dc.asdict(self)
        del d["sign"]
        return json.dumps(d)
    
    def json_dumps(self) -> str:
        return json.dumps(dc.asdict(self))
    @classmethod
    def json_loads(cls, string) -> Transaction:
        return cls(**json.loads(string))

In [0]:
pip install Crypto


In [0]:
pip install PyCrypto

     |████████████████████████████████| 450kB 9.5MB/s 
  Created wheel for PyCrypto: filename=pycrypto-2.6.1-cp36-cp36m-linux_x86_64.whl size=499957 sha256=dee4d62d0a52a93ce91ad9fd30139c6a7ff61396039f8c560d2b414f4a5b5c74
  Stored in directory: /root/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
Successfully built PyCrypto


In [0]:
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
import binascii

In [0]:
def decode_key(key):
    if isinstance(key, str):
        return key
    return binascii.hexlify(key.exportKey(format='DER')).decode('ascii')
def encode_key(key):
    if isinstance(key, RSA._RSAobj):
        return key
    return RSA.importKey(binascii.unhexlify(key))

In [0]:
class Wallet:
    def __init__(self):
        key=RSA.generate(1024)
        self.private_key = decode_key(key)
        self.address = decode_key(key.publickey())
    
    def sign_transaction(self, transaction) -> Transaction:
        # generate signer from self private key
        signer=PKCS1_v1_5.new(encode_key(self.private_key))
        # hash the transaction
        h=SHA.new(transaction.str_data().encode())
        # add signature to the transaction, and return it
        return dc.replace(transaction, sign=signer.sign(h).hex())
    
    def send(self, receiver_address, value) -> Transaction:
        """Generate transaction, and add signature"""
        transaction=Transaction(self.address, receiver_address, value)
        return self.sign_transaction(transaction)

In [0]:
def verify_transaction(transaction) -> bool:
    if transaction.sign is None:
        return False
    # hash the transaction
    h=SHA.new(transaction.str_data().encode())
    # generate verifier with public key
    verifier=PKCS1_v1_5.new(encode_key(transaction.sender_address))
    # is the signature correct?
    return verifier.verify(h, binascii.unhexlify(transaction.sign))

In [0]:
alice=Wallet() # now the address is automatically generated
bob=Wallet()
ledger=Ledger()

In [0]:
transaction=alice.send(bob.address, 5)
# Bank append the transaction to its ledger
ledger.append(transaction)

In [0]:
verify_transaction(transaction)

True

In [0]:
transaction.value=7
verify_transaction(transaction)

False

In [0]:
from typing import Tuple, Sequence
import hashlib

In [0]:
@dataclass
class Block:
    time: float
    transactions: Tuple[Transaction]
    previous_hash: str
    sign: str = None
        
    def json_dumps(self) -> str:
        dct=dc.asdict(self)
        dct["transactions"]=[t.json_dumps() for t in self.transactions]
        return json.dumps(dct)
    @classmethod
    def json_loads(cls, string) -> str:
        dct=json.loads(string)
        dct["transactions"]=tuple([Transaction.json_loads(t) for t in dct["transactions"]])
        return cls(**dct)
        
    def hash(self) -> str: 
        block_bytes=self.json_dumps().encode()
        return hashlib.sha256(block_bytes).hexdigest()

In [0]:
from time import time

In [0]:
class TimestampServer:
    def __init__(self):
        key=RSA.generate(1024)
        self.public_key=key.publickey()
        self.signer=PKCS1_v1_5.new(key)
        
        genesis=Block(time(), (), "0")
        self.block_chain=[genesis]
        
    def generate_block(self, transactions: Sequence[Transaction]):
        # generate block
        block=Block(time(), tuple(transactions), self.block_chain[-1].hash())
        
        # sign the block
        dct=dc.asdict(block)
        del dct["sign"]
        block.sign=self.signer.sign(SHA.new(json.dumps(dct).encode())).hex()
        
        # publish the block
        self.block_chain.append(block)

In [0]:
def verify_block(previous_block, block, timestamp_server_publickey) -> bool:
    is_correct_hash = previous_block.hash() ==  block.previous_hash
    is_correct_transactions = all(map(verify_transaction, block.transactions))
    
    dct=dc.asdict(block)
    del dct["sign"]
    h=SHA.new(json.dumps(dct).encode())
    verifier=PKCS1_v1_5.new(timestamp_server_publickey)
    is_correct_sign=verifier.verify(h, binascii.unhexlify(block.sign))
    return is_correct_hash and is_correct_transactions and is_correct_sign

In [0]:
def verify_blockchain(chain, timestamp_server_publickey):
    for i in range(len(chain)-1):
        if not verify_block(chain[-i-2],chain[-i-1],timestamp_server_publickey):
            return False
    return True

In [0]:
timestamp_server=TimestampServer()

In [0]:
transactions=[]
transactions.append(alice.send(bob.address, 5))
transactions.append(bob.send(alice.address, 7))

In [0]:
timestamp_server.generate_block(transactions)

In [0]:
verify_blockchain(timestamp_server.block_chain, timestamp_server.public_key)

True

In [0]:
@dataclass
class Block:
    time: float
    transactions: Tuple[Transaction]
    previous_hash: str
    nonce: int = None
        
    def json_dumps(self) -> str:
        dct=dc.asdict(self)
        dct["transactions"]=[t.json_dumps() for t in self.transactions]
        return json.dumps(dct)
    
    @classmethod
    def json_loads(cls, string) -> str:
        dct=json.loads(string)
        dct["transactions"]=tuple([Transaction.json_loads(t) for t in dct["transactions"]])
        return cls(**dct)
        
    def hash(self): 
        block_bytes=self.json_dumps().encode()
        return hashlib.sha256(block_bytes).hexdigest()

In [0]:
difficulty=3
def valid_proof(block):
    return block.hash()[:difficulty] == "0" * difficulty

In [0]:
def mine(block):
    nonce=0
    block.nonce=nonce
    while not valid_proof(block):
        nonce += 1
        block.nonce=nonce
    return block

In [0]:
def verify_block(previous_block, block) -> bool:
    is_correct_hash = previous_block.hash() ==  block.previous_hash
    is_correct_transactions = all(map(verify_transaction, block.transactions))
    is_correct_proof = valid_proof(block)
    return is_correct_hash and is_correct_transactions and is_correct_proof

In [0]:
class BlockChain(list):
    def json_dumps(self) -> str:
        return json.dumps([block.json_dumps() for block in self])
    @classmethod
    def json_loads(cls, string) -> str:
        return cls([Block.json_loads(s) for s in json.loads(string)])

In [0]:
genesis=Block(time(), (), "0")
block_chain=BlockChain([genesis])

# 取引
transactions=[]
transactions.append(alice.send(bob.address, 5))
transactions.append(bob.send(alice.address, 5))

In [0]:
previous_hash=block_chain[-1].hash()
block=mine(Block(time(), tuple(transactions), previous_hash))


In [0]:
block.nonce

2236

In [0]:
verify_block(block_chain[-1], block)

True

In [0]:
from uuid import uuid4
from time import sleep
from copy import deepcopy
import multiprocessing as mp
import random
from itertools import count

In [0]:
difficulty=3
class Node:
    def __init__(self, network, genesis, uuid=None):
        self.chain=BlockChain([genesis])
        self.uuid=uuid or str(uuid4())
        self.network=network
        
    def work(self, verbose=False):
        while True:
            if not self.mine():
                sleep(0.1) # wait due to no transactions
            else:
                if verbose: print(self.uuid, "Mined a block")
            if self.add_block():
                if verbose: print(self.uuid, "Added one block")
            self.network.post_chain(self.chain, self.uuid) # publish my blockchain
            if self.resolve_conflicts():
                if verbose: print(self.uuid, "Change chain")
    
    def mine(self):
        transactions=self.network.get_transactions(self.uuid)
        if len(transactions)==0:
            return False # cannot mine due to no transactions
        previous_hash=self.chain[-1].hash()
        block=Block(time(), tuple(transactions), previous_hash)
        
        for i, n in enumerate(self.random_generator()):
            block.nonce=n
            if self.verify_proof(block):
                break
                
            if i % 100 == 0 and self.add_block(): # someone mined a block
                return True
            if i % 100 == 0 and self.resolve_conflicts(): # someone mined a block
                return True
                
        self.network.broadcast_block(block, self.uuid)
        return True # successfully mined
    
    @staticmethod
    def random_generator(step=None):
        step = step or random.randint(1e4, 1e5)
        for c in count():
            for i in random.sample(range(c*step, (c+1)*step), step):
                yield i
        
    def add_block(self):
        for block in self.network.get_blocks(self.uuid):
            if self.verify_block(block):
                self.chain.append(block)
                if self.verify_chain(self.chain):
                    return True 
                else:
                    self.chain.pop(-1)
        return False # no block is added
    
    def resolve_conflicts(self):
        """Longest valid chain is authoritative"""
        authoritative_chain=self.chain
        for chain in self.network.get_neighbour_chains(self.uuid):
            if not self.verify_chain(chain):
                # node is incorrect
                continue
            if len(chain)>len(authoritative_chain):
                # Longest valid chain is authoritative
                authoritative_chain=deepcopy(chain)
        self.chain=authoritative_chain
        return self.chain is not authoritative_chain
        
    @staticmethod
    def verify_transaction(transaction):
        if transaction.sign is None:
            return False
        h=SHA.new(transaction.str_data().encode())
        verifier=PKCS1_v1_5.new(encode_key(transaction.sender_address))
        return verifier.verify(h, binascii.unhexlify(transaction.sign))
    
    @staticmethod
    def verify_proof(block):
        return block.hash()[:difficulty] == "0" * difficulty
    
    def verify_block(self, block) -> bool:
        is_correct_transactions = all(map(self.verify_transaction, block.transactions))
        is_correct_proof = self.verify_proof(block)
        return is_correct_transactions and is_correct_proof
    
    def verify_chain(self, chain):
        for i in range(len(chain)-1, 0, -1):
            if not self.verify_block(chain[i]):
                return False
            if chain[i-1].hash() != chain[i].previous_hash:
                return False
        return True

In [0]:
class Wallet:
    def __init__(self, network, nodes=None):
        key=RSA.generate(1024)
        self.private_key = decode_key(key)
        self.address = decode_key(key.publickey())
        self.network=network
        self.nodes=nodes or []
    
    def sign_transaction(self, transaction):
        signer=PKCS1_v1_5.new(encode_key(self.private_key))
        h=SHA.new(transaction.str_data().encode())
        return dc.replace(transaction, sign=signer.sign(h).hex())
    
    def send(self, receiver_address, value):
        transaction=Transaction(self.address, receiver_address, value)
        self.broadcast(self.sign_transaction(transaction))
    
    def broadcast(self, transaction):
        for uuid in self.nodes:
            self.network.post_transaction(transaction, uuid)

In [0]:
from typing import List

In [0]:
class Network:
    def __init__(self, neighbours=()):
        self.manager=mp.Manager()
        self.chains=self.manager.dict()
        self.blocks=self.manager.dict()
        self.transactions=self.manager.dict()
        self.neighbours=dict(neighbours)
        
    def post_chain(self, chain, uuid):
        self.chains[uuid]=chain.json_dumps()
    
    def get_chain(self, uuid) -> BlockChain:
        if uuid not in self.chains:
            return []
        return BlockChain.json_loads(self.chains[uuid])
    
    def get_neighbour_chains(self, uuid) -> List[BlockChain]:
        return [self.get_chain(neigh) for neigh in self.neighbours[uuid] if neigh != uuid]
    
    def post_block(self, block, uuid):
        if uuid not in self.blocks:
            self.blocks[uuid]=self.manager.list([block.json_dumps()])
        else:
            self.blocks[uuid].append(block.json_dumps())
            
    def broadcast_block(self, block, uuid):
        for receiver in self.neighbours[uuid]:
            self.post_block(block, receiver)
            
    def get_blocks(self, uuid) -> List[Block]:
        if uuid not in self.blocks:
            return []
        res=[Block.json_loads(s) for s in self.blocks[uuid]]
        self.blocks[uuid][:]=[]
        return res
    
    def post_transaction(self, transaction, uuid):
        if uuid not in self.transactions:
            self.transactions[uuid]=self.manager.list([transaction.json_dumps()])
        else:
            self.transactions[uuid].append(transaction.json_dumps())
            
    def get_transactions(self, uuid) -> Tuple[Transaction]:
        if uuid not in self.transactions:
            return []
        res=[Transaction.json_loads(s) for s in self.transactions[uuid]]
        self.transactions[uuid][:]=[]
        return res
    
    def shutdown(self):
        self.manager.shutdown()

ノード1つの場合のテスト

In [0]:
genesis=Block(time(), (), "0")
uuid="test"
network=Network({uuid:[uuid]})
node=Node(network, genesis, uuid)

In [0]:
alice=Wallet(network, [uuid])
bob=Wallet(network, [uuid])

In [0]:
alice.send(bob.address, 5)

In [0]:
network.shutdown()

デモ: 分散blockchainのデモ

multiprocessingでnetworkのデモを行う

In [0]:
genesis=Block(time(), (), "0")
network=Network()

node生成
適当にnum_nodesを変更して遊ぶと良い

In [0]:
num_nodes=7
nodes=[str(uuid4()) for _ in range(num_nodes)]

nodeのnetworkの状態を定義する
とりあえずは全結合にする
つまり全てのNode同士，直接やりとりできる

In [0]:
for uuid in nodes:
    network.neighbours[uuid]=nodes

multiprocess workerを作って起動する

In [0]:
def node_work(uuid):
    node=Node(network, genesis, uuid)
    node.work()

In [0]:
workers=[]
for uuid in nodes:
    worker=mp.Process(target=node_work, args=(uuid,))
    workers.append(worker)
    worker.start()

peopleを生成する
これも適当に人数を変えて遊ぶと良い

In [0]:
num_people=5
people=[Wallet(network, random.sample(nodes, random.randint(1, len(nodes)))) for _ in range(num_people)]

blockchain の表示

In [0]:
from IPython.display import clear_output
def draw_chains(network):
    while True:
        line=""
        for k, v in network.chains.items():
            line += f"(Node: {k[:4]}) "
            chain=BlockChain.json_loads(v)
            h=list(map(lambda x: x.hash()[:5], chain))
            line += "--".join(map(lambda x: f"[{x}]", h))
            line += "\n"
        print(line.rstrip())
        clear_output(True)
        sleep(0.1)

worker=mp.Process(target=draw_chains, args=(network,))
worker.start()

(Node: e3e7) [502b6]--[0004a]--[000cf]--[000cc]--[00076]--[0006c]--[000af]--[000ca]--[000aa]
(Node: 6d2b) [502b6]--[0004a]--[000cf]--[000cc]--[00076]--[0006c]--[000af]--[000ca]--[000aa]
(Node: 2bad) [502b6]--[0004a]--[000cf]--[000cc]--[00076]--[0006c]--[000af]--[000ca]--[000aa]
(Node: ec26) [502b6]--[0004a]--[000cf]--[000cc]--[00076]--[0006c]--[000af]--[000ca]--[000aa]
(Node: 27c2) [502b6]--[0004a]--[000cf]--[000cc]--[00076]--[0006c]--[000af]--[000ca]--[000aa]
(Node: f7a7) [502b6]--[0004a]--[000cf]--[000cc]--[00076]--[0006c]--[000af]--[000ca]--[000aa]
(Node: 1edd) [502b6]--[0004a]--[000cf]--[000cc]--[00076]--[0006c]--[000af]--[000ca]--[000aa]


取引をして遊ぶ
ランダムに取引が行われるコードを書いてみる
上の表示が変化する!

In [0]:
def generate_transactions():
    for _ in range(random.randint(1,10)):
        sender, receiver= random.sample(people, 2)
        sender.send(receiver.address, random.randint(1, 100))

In [0]:
generate_transactions()

networkをちょと変えて遊んで見る
ちょっと断線させる

In [0]:
for uuid in nodes:
    network.neighbours[uuid]=random.sample(nodes, 3)

In [0]:
generate_transactions()


片付け

In [0]:
worker.terminate()
worker.kill()
for worker in workers:
    worker.terminate()
    worker.kill()
network.shutdown()